In [613]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [614]:
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning) # FutureWarning 제거

In [615]:
# 국회의원 298명에 대한 정보를 담고 있는 member df, node가 될 예정
member = pd.read_excel('./raw_data/member_of_Congress.xlsx')
member.head(1)

,이름,한자명,영문명칭,음/양력,생년월일,정당명,선거구,대표 위원회,재선,당선,...,이메일,홈페이지,보좌관,비서관,비서,재직구분,직업,학력,경력,득표수
0,강기윤,姜起潤,KANG GIYUN,양,1960-06-04,국민의힘,경남 창원시성산구,보건복지위원회,재선,"제19대, 제21대",...,ggotop@naver.com,http://blog.naver.com/ggotop,한영애,"김샛별, 장원종","안효상, 빈자영, 이유진, 김지훈, 조옥자",60001,정당인,창원대학교 대학원 졸업(행정학 박사),(전)19대 국회의원\n(현)미래통합당 경남도당 민생위원회 위원장,"61,782\n(47.30)"


In [616]:
# member df 전처리 과정
member.loc[member['이름'] == '양원영', ['이름']] = '양이원영' # 이름 오류 수정
member['이름'] = member['이름'] + '(' + member['한자명'] + ')' # primary key인 이름(한자) 생성
member.sort_values(by = '이름', inplace = True) # 동명이인의 한자명에 따른 정렬 순서 이상 때문에 정렬 한 번
member.reset_index(drop = True, inplace = True) # 정렬에 따른 인덱스 이상 때문에 인덱스 정렬 한 번
member = member[['이름','생년월일','정당명','선거구','재선','소속 위원회 목록','성별','학력']]
member.head(3) # 필요 없는 데이터 잠깐 지우겠슴다

,이름,생년월일,정당명,선거구,재선,소속 위원회 목록,성별,학력
0,강기윤(姜起潤),1960-06-04,국민의힘,경남 창원시성산구,재선,보건복지위원회,남,창원대학교 대학원 졸업(행정학 박사)
1,강대식(姜大植),1959-11-02,국민의힘,대구 동구을,초선,국토교통위원회,남,영남대학교 대학원 경영학과 졸업(경영학박사)
2,강득구(姜得求),1963-05-27,더불어민주당,경기 안양시만안구,초선,"인구위기특별위원회, 교육위원회",남,연세대학교 행정대학원 졸업(행정학석사)


In [617]:
# 그래프에서 feature로 활용하기 위해 당선횟수를 숫자로
member.rename(columns = {'재선' : '당선'}, inplace = True)
member['당선'] = member['당선'].str.replace('초', '1')
member['당선'] = member['당선'].str.replace('재', '2')
member['당선'] = member['당선'].str.replace('선', '')
member['당선'] = member['당선'].apply(pd.to_numeric)
member['당선'].value_counts()

1    155
2     69
3     41
4     20
5     12
6      1
Name: 당선, dtype: int64

In [618]:
# 그래프에서 feature로 활용하기 위해 생년월일 to 나이
from datetime import datetime

member['생년월일'] = member['생년월일'].str.replace('-', '')
member['생년월일'] = pd.to_numeric(member['생년월일'])
today = int(datetime.now().strftime("%Y%m%d"))
member['생년월일'] = (today - member['생년월일'])//10000
member.rename(columns = {'생년월일' : '나이'}, inplace = True)
member.head(3)

,이름,나이,정당명,선거구,당선,소속 위원회 목록,성별,학력
0,강기윤(姜起潤),63,국민의힘,경남 창원시성산구,2,보건복지위원회,남,창원대학교 대학원 졸업(행정학 박사)
1,강대식(姜大植),64,국민의힘,대구 동구을,1,국토교통위원회,남,영남대학교 대학원 경영학과 졸업(경영학박사)
2,강득구(姜得求),60,더불어민주당,경기 안양시만안구,1,"인구위기특별위원회, 교육위원회",남,연세대학교 행정대학원 졸업(행정학석사)


In [619]:
# 그래프에서 feature로 활용하기 위해 학력 표준화
arr = []
for edu in member['학력']:
  edu = str(edu)
  if '박사' in edu and '수료' in edu:
    arr.append('석사')
  elif '박사' in edu:
    arr.append('박사')
  elif '수료' in edu and '석사' in edu:
    arr.append('학사')
  elif '석사' in edu:
    arr.append('석사')
  elif '중퇴' in edu or '고등학교' in edu:
    arr.append('고졸')
  else:
    arr.append('학사')

member['학력'] = arr
member['학력'].value_counts()

학사    128
석사    102
박사     66
고졸      2
Name: 학력, dtype: int64

In [620]:
# 그래프에서 feature로 활용하기 위해 위원회 정리...를 어떻게 할까
committee = member['소속 위원회 목록'][member['소속 위원회 목록'].notnull()].str.split(', ')
comm = sorted(list(set([data for inner_list in committee for data in inner_list])))
print(comm, sep = ' ')

['과학기술정보방송통신위원회', '교육위원회', '국방위원회', '국토교통위원회', '국회 2030 부산세계박람회 유치지원 특별위원회', '국회운영위원회', '기획재정위원회', '기후위기특별위원회', '농림축산식품해양수산위원회', '문화체육관광위원회', '법제사법위원회', '보건복지위원회', '산업통상자원중소벤처기업위원회', '여성가족위원회', '연금개혁특별위원회', '예산결산특별위원회', '외교통일위원회', '윤리특별위원회', '인구위기특별위원회', '정무위원회', '정보위원회', '정치개혁특별위원회', '첨단전략산업특별위원회', '행정안전위원회', '헌법재판소장(이종석)임명동의에관한인사청문특별위원회', '환경노동위원회']


In [621]:
idx_temp = member[member['소속 위원회 목록'].isna()].index[0]
for i in member.index:
    if i != idx_temp:
        member['위원회'] = member.loc[i, '소속 위원회 목록'].count(',') + 1
member.loc[idx_temp]['위원회'] = 0
member.head(3)

C:\Users\parkg\AppData\Local\Temp\ipykernel_22104\64595070.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member.loc[idx_temp]['위원회'] = 0


,이름,나이,정당명,선거구,당선,소속 위원회 목록,성별,학력,위원회
0,강기윤(姜起潤),63,국민의힘,경남 창원시성산구,2,보건복지위원회,남,박사,1
1,강대식(姜大植),64,국민의힘,대구 동구을,1,국토교통위원회,남,박사,1
2,강득구(姜得求),60,더불어민주당,경기 안양시만안구,1,"인구위기특별위원회, 교육위원회",남,석사,1


In [622]:
# 가장 문제인 법안을 함께 작업한 국회의원 state
relation_raw = pd.read_csv('./raw_data/scraped_data_3.csv')
relation_raw.head(3)

,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,이용우,더불어민주당,李龍雨,9771184
0,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,강훈식,더불어민주당,姜勳植,9771007.0
1,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,김상희,더불어민주당,金相姬,9770527.0
2,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,김한규,더불어민주당,金翰奎,9771296.0


In [623]:
# 컬럼명이 없어서 전처리
tmp1 = ['법안 ID','이름','정당명','한자명','번호']
tmp2 = list(relation_raw.columns)
tmp3 = pd.DataFrame([tmp2], columns = tmp1)
relation_raw.columns = tmp3.columns.copy()
relation = pd.concat([tmp3, relation_raw], axis = 0)
relation.reset_index(drop = True, inplace = True)
relation.dropna(subset = ['이름','한자명'], inplace = True) # 이름이 없는 데이터는 살릴 수 없어 과감히 drop
relation.head(3)

,법안 ID,이름,정당명,한자명,번호
0,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,이용우,더불어민주당,李龍雨,9771184
1,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,강훈식,더불어민주당,姜勳植,9771007.0
2,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,김상희,더불어민주당,金相姬,9770527.0


In [624]:
# primary key인 이름(한자) 생성
relation['이름'] = relation['이름'] + '(' + relation['한자명'] + ')'
relation.drop(columns = ['한자명'], inplace = True)
relation.drop_duplicates(inplace = True) # 중복 데이터를 제거
relation.head(3)

,법안 ID,이름,정당명,번호
0,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,이용우(李龍雨),더불어민주당,9771184
1,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,강훈식(姜勳植),더불어민주당,9771007.0
2,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,김상희(金相姬),더불어민주당,9770527.0


In [625]:
# relation에는 지금은 없는 의원과 이름이 잘못된 의원이 존재
name_arr1 = sorted(relation['이름'].unique())
name_arr2 = sorted(list(member['이름'].copy()))
name_arr3 = []
for i in name_arr1:
  if i not in name_arr2:
    name_arr3.append(i)
print(f'법안의 의원 명수: {len(name_arr1)}. 실제 의원 명수: {len(name_arr2)}. 법안에 있는 의원이 아닌 사람 수: {len(name_arr3)}')

법안의 의원 명수: 323. 실제 의원 명수: 298. 법안에 있는 의원이 아닌 사람 수: 26


In [626]:
# 이름 고쳐줄 의원들 딕셔너리
mem_dict = {}
for i in name_arr2:
  for j in name_arr3:
    if i[0:3] == j[0:3]:
      mem_dict[j] = i
print(mem_dict, sep = ' ')

{'고민정(IOUAiO)': '고민정(高旼廷)', '고용진(高榕진)': '고용진(高榕禛)', '김종민(金鍾民)': '김종민(金鐘民)', '박정하(朴正何)': '박정하(朴正河)', '심상정(沈相정)': '심상정(沈相奵)', '이종성(李鐘成)': '이종성(李鍾成)', '지성호(地成浩)': '지성호(池成浩)'}


In [627]:
ban_list = [] # 지금은 없어서 제거해줄 의원들 리스트
for i in name_arr3:
  if i not in mem_dict:
    ban_list.append(i)
print(ban_list, sep = ' ')

['곽상도(郭尙道)', '김선교(金善敎)', '김은혜(金恩慧)', '김진애(金鎭愛)', '김태흠(金泰欽)', '박완수(朴完洙)', '송영길(宋永吉)', '오영훈(吳怜勳)', '윤희숙(尹喜淑)', '이광재(李光宰)', '이규민(李圭閔)', '이낙연(李洛淵)', '이상직(李相稷)', '이영(李永)', '정정순(鄭正淳)', '정찬민(鄭燦敏)', '조태용(趙太庸)', '최강욱(崔康旭)', '홍준표(洪準杓)']


In [628]:
relation.reset_index(inplace = True, drop = True)
for i in range(len(relation)): # 이름 잘못된 거 처리
  if relation['이름'][i] in mem_dict:
      relation['이름'][i] = mem_dict[relation['이름'][i]]
relation['이름'].value_counts()

오영환(吳永煥)    3095
양정숙(梁貞淑)    2926
김정호(金禎鎬)    2826
김병욱(金炳旭)    2820
이용빈(李龍彬)    2678
            ... 
이낙연(李洛淵)     125
박병석(朴炳錫)      89
이재명(李在明)      87
강성희(姜聖熙)      48
허숙정(許淑湞)       9
Name: 이름, Length: 316, dtype: int64

In [629]:
relation.set_index('이름', inplace = True) # 인덱스로 처리하면 속도가 빠르다
relation.drop(ban_list, axis = 0, inplace = True) # 없는 의원들 제거
relation.reset_index(inplace = True)
relation['이름'].value_counts()

오영환(吳永煥)    3095
양정숙(梁貞淑)    2926
김정호(金禎鎬)    2826
김병욱(金炳旭)    2820
이용빈(李龍彬)    2678
            ... 
최재형(崔在亨)     131
박병석(朴炳錫)      89
이재명(李在明)      87
강성희(姜聖熙)      48
허숙정(許淑湞)       9
Name: 이름, Length: 297, dtype: int64

In [630]:
relation.set_index('이름', inplace = True)
for i in member['이름']:
    if len(relation.loc[i]['정당명'].unique()) != 1:
        print(i, end = ' ')

강기윤(姜起潤) 강대식(姜大植) 강민국(姜旻局) 강민정(姜旼姃) 구자근(具滋根) 권명호(權明浩) 권성동(權性東) 권영세(權寧世) 권은희(權垠希) 김기현(金起炫) 김남국(金南局) 김도읍(金度邑) 김미애(金美愛) 김병욱(金炳旭) 김병욱(金炳旭) 김상훈(金相勳) 김석기(金碩基) 김성원(金成願) 김승수(金承洙) 김영식(金英植) 김예지(金睿智) 김용판(金用判) 김웅(金雄) 김의겸(金宜謙) 김정재(金汀才) 김진표(金振杓) 김태호(金台鎬) 김형동(金亨東) 김홍걸(金弘傑) 김희곤(金熙坤) 김희국(金熙國) 류성걸(柳性杰) 민형배(閔馨培) 박대수(朴大壽) 박대출(朴大出) 박덕흠(朴德欽) 박병석(朴炳錫) 박성민(朴聖敏) 박성중(朴成重) 박수영(朴洙瑩) 박완주(朴完柱) 박진(朴振) 박형수(朴亨修) 배준영(裵俊英) 배현진(裵賢鎭) 백종헌(白宗憲) 서범수(徐範洙) 서병수(徐秉洙) 서일준(徐一俊) 서정숙(徐正淑) 성일종(成一鍾) 송석준(宋錫俊) 송언석(宋彦錫) 신원식(申源湜) 안병길(安炳吉) 양금희(梁琴喜) 양이원영(梁李媛瑛) 양향자(梁香子) 엄태영(嚴泰永) 유경준(兪京濬) 유상범(劉相凡) 유의동(兪義東) 윤관석(尹官石) 윤두현(尹斗鉉) 윤미향(尹美香) 윤상현(尹相現) 윤영석(尹永碩) 윤재옥(尹在玉) 윤주경(尹柱卿) 윤창현(尹暢賢) 윤한홍(尹漢洪) 이달곤(李達坤) 이만희(李晩熙) 이명수(李明洙) 이성만(李成萬) 이양수(李亮壽) 이용(李鏞) 이용호(李容鎬) 이종배(李鍾培) 이종성(李鍾成) 이주환(李周桓) 이채익(李埰益) 이철규(李喆圭) 이태규(李泰珪) 이헌승(李憲昇) 임병헌(林炳憲) 임이자(林利子) 장제원(張濟元) 전봉민(田奉珉) 전주혜(全珠惠) 정경희(丁慶姬) 정동만(鄭東萬) 정운천(鄭雲天) 정점식(鄭点植) 정진석(鄭鎭碩) 정희용(鄭熙溶) 조경태(趙慶泰) 조명희(曺明姬) 조수진(趙修眞) 조해진(曺海珍) 주호영(朱豪英) 지성호(池成浩) 최승재(崔承宰) 최연숙(崔姸淑) 최춘식(崔春植) 최형두(崔炯斗) 추경호(秋慶鎬) 태영호(太永浩) 하영제(河榮帝) 하태경(河泰慶) 한기호(韓起鎬) 한무경(韓

In [631]:
member[member['이름'].duplicated(keep = False)]

,이름,나이,정당명,선거구,당선,소속 위원회 목록,성별,학력,위원회
34,김병욱(金炳旭),58,더불어민주당,경기 성남시분당구을,2,"국토교통위원회, 예산결산특별위원회",남,석사,1
35,김병욱(金炳旭),46,국민의힘,경북 포항시남구울릉군,1,과학기술정보방송통신위원회,남,석사,1


In [632]:
idx_temp = member[member['이름'].duplicated(keep = False)].index
name_temp1 = member.loc[idx_temp[0], '이름']
member.loc[idx_temp[0], '이름'] = name_temp1[0:3] + 'A' + name_temp1[3:]
name_temp1 = member.loc[idx_temp[1], '이름']
member.loc[idx_temp[1], '이름'] = name_temp1[0:3] + 'B' + name_temp1[3:]
member.loc[idx_temp]

,이름,나이,정당명,선거구,당선,소속 위원회 목록,성별,학력,위원회
34,김병욱A(金炳旭),58,더불어민주당,경기 성남시분당구을,2,"국토교통위원회, 예산결산특별위원회",남,석사,1
35,김병욱B(金炳旭),46,국민의힘,경북 포항시남구울릉군,1,과학기술정보방송통신위원회,남,석사,1


In [633]:
relation.reset_index(inplace = True)
relation.loc[(relation['이름'] == '김병욱(金炳旭)') & (relation['정당명'] == '더불어민주당'), '이름'] = '김병욱A(金炳旭)'
relation.loc[(relation['이름'] == '김병욱(金炳旭)') & (relation['정당명'] == '무소속'), '정당명'] = '국민의힘'
relation.loc[(relation['이름'] == '김병욱(金炳旭)') & (relation['정당명'] == '미래통합당'), '정당명'] = '국민의힘'
relation.loc[(relation['이름'] == '김병욱(金炳旭)') & (relation['정당명'] == '국민의힘'), '이름'] = '김병욱B(金炳旭)'
relation.loc[relation['이름'] == '김병욱(金炳旭)'].head(3)

,이름,법안 ID,정당명,번호


In [634]:
relation.set_index('이름', inplace = True)
for i in range(len(member)):
    relation.loc[member.loc[i, '이름'], '정당명'] = member.loc[i, '정당명']
relation.head(3)

,법안 ID,정당명,번호
이름,,,
이용우(李龍雨),PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,더불어민주당,9771184
강훈식(姜勳植),PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,더불어민주당,9771007.0
김상희(金相姬),PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,더불어민주당,9770527.0


In [635]:
for i in member['이름']:
    if len(relation.loc[i]['정당명'].unique()) != 1:
        print(i, end = ' ')

In [636]:
# 아직은 상태 feature만을 가져오기 위한 state
custom_column_names = ['제목','담당','날짜','임시','임시2','임시3','상태','계류','법안 ID']
state = pd.read_csv('./raw_data/scraped_data.csv', names = custom_column_names, skiprows = 1)
state.dropna(subset = ['상태'], inplace = True) # 상태가 없으면 쓸모가 없지요
state[state['상태'] == '본희의의결'] = '본회의의결'
state.head(3)

,제목,담당,날짜,임시,임시2,임시3,상태,계류,법안 ID
2125205,자본시장과 금융투자업에 관한 법률 일부개정법률안(이용우의원 등 11인),의원,2023-10-30,NaN,NaN,NaN,접수,계,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3
2125204,공유재산 및 물품 관리법 일부개정법률안(황희의원 등 12인),의원,2023-10-30,NaN,NaN,NaN,접수,계,PRC_O2T3B1B0Z2Z4F1E3C3Z9Y4H8G7F4M3
2125203,학교용지 확보 등에 관한 특례법 일부개정법률안(황희의원 등 12인),의원,2023-10-30,NaN,NaN,NaN,접수,계,PRC_Z2X3W1X0W2R4Q1Y3X4W1V3D3B6I0G7


In [637]:
good = ['공포', '본회의의결', '수정안반영폐기', '대안반영폐기']
bad = ['폐기', '정부이송', '본회의불부의', '체계자구심사', '소관위심사', '소관위접수', '철회', '본회의부의안건', '접수']

def change_state(x) :
  if str(x) in good:
    return '가결'
  else:
    return '가결 외'
  
state['상태'] = state['상태'].apply(change_state)
state['상태'].value_counts()

가결 외    17708
가결       7665
Name: 상태, dtype: int64

In [638]:
# join 해주기 위해 전처리
state_temp = state.loc[:, ['법안 ID','상태','제목']]
state_temp.reset_index(inplace = True)
state_temp.drop(columns = ['index'], inplace = True)
state_temp.head(3)

,법안 ID,상태,제목
0,PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,가결 외,자본시장과 금융투자업에 관한 법률 일부개정법률안(이용우의원 등 11인)
1,PRC_O2T3B1B0Z2Z4F1E3C3Z9Y4H8G7F4M3,가결 외,공유재산 및 물품 관리법 일부개정법률안(황희의원 등 12인)
2,PRC_Z2X3W1X0W2R4Q1Y3X4W1V3D3B6I0G7,가결 외,학교용지 확보 등에 관한 특례법 일부개정법률안(황희의원 등 12인)


In [639]:
# inner join
relation.reset_index(inplace = True)
relation_state = pd.merge(relation, state_temp, how = 'inner', on = '법안 ID')
relation_state.drop_duplicates(inplace = True) # 중복 데이터를 제거
relation_state.reset_index(drop = True, inplace = True)
relation_state.head(3)

,이름,법안 ID,정당명,번호,상태,제목
0,이용우(李龍雨),PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,더불어민주당,9771184,가결 외,자본시장과 금융투자업에 관한 법률 일부개정법률안(이용우의원 등 11인)
1,강훈식(姜勳植),PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,더불어민주당,9771007.0,가결 외,자본시장과 금융투자업에 관한 법률 일부개정법률안(이용우의원 등 11인)
2,김상희(金相姬),PRC_M2L3N1N0M2L5J1S4Q3R7Q3O5W3V5U3,더불어민주당,9770527.0,가결 외,자본시장과 금융투자업에 관한 법률 일부개정법률안(이용우의원 등 11인)


In [640]:
tm = pd.DataFrame(relation_state.groupby(['이름','상태'])['법안 ID'].count())
tm.reset_index(inplace = True)
tm.set_index(['이름'], inplace = True)
mem_law = pd.DataFrame(columns = ['이름','가결','가결 외'])
mem_law['이름'] = member['이름'].copy()
mem_law.set_index(['이름'], inplace = True)
for i in member['이름']:
    mem_law.loc[i][tm.loc[i]['상태'][0]] = tm.loc[i]['법안 ID'][0]
    mem_law.loc[i][tm.loc[i]['상태'][1]] = tm.loc[i]['법안 ID'][1]
mem_law.reset_index(inplace = True)
mem_law.head(3)

,이름,가결,가결 외
0,강기윤(姜起潤),343,893
1,강대식(姜大植),306,727
2,강득구(姜得求),387,1193


In [641]:
num = len(member)
myArr = np.zeros((num*(num-1)//2, 4), dtype = int)
df_graph = pd.DataFrame(myArr, columns = ['의원A', '의원B', '가결', '가결 외'])
df_graph.head(3)

,의원A,의원B,가결,가결 외
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0


In [642]:
# 파이썬스러운 날먹 방법이 분명 있을 것 같은데 생각이 안 난다...
df_graph = df_graph.astype({'의원A':'object', '의원B':'object'})
cnt = 0
for i in range(num):
  for j in range(i + 1, num):
    df_graph['의원A'][cnt], df_graph['의원B'][cnt] = member['이름'][i], member['이름'][j]# df_graph에 의원 이름을 인덱스로 넣어주기
    cnt+=1
df_graph.set_index(['의원A', '의원B'], inplace = True) # 속도를 위해 의원 이름을 index로
df_graph.head(3)

C:\Users\parkg\AppData\Local\Temp\ipykernel_22104\4218622639.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph['의원A'][cnt], df_graph['의원B'][cnt] = member['이름'][i], member['이름'][j]# df_graph에 의원 이름을 인덱스로 넣어주기


가결  가결 외
의원A      의원B               
강기윤(姜起潤) 강대식(姜大植)   0     0
         강득구(姜得求)   0     0
         강민국(姜旻局)   0     0

In [643]:
idx = [0] # 한 법안의 index, idx[i]:idx[i+1]에는 같은 법안을 낸 여러 사람의 명단이 있다.
for i in range(len(relation_state)-1):
  if relation_state['법안 ID'][i] != relation_state['법안 ID'][i+1]:
    idx.append(i+1)
idx.append(len(relation_state))
print(idx, sep = ' ')

[0, 11, 23, 35, 47, 59, 70, 80, 91, 103, 113, 124, 135, 145, 160, 173, 183, 193, 203, 214, 224, 235, 246, 256, 282, 292, 309, 319, 331, 341, 351, 364, 375, 556, 566, 576, 586, 596, 606, 616, 628, 638, 648, 658, 668, 679, 689, 702, 713, 723, 735, 745, 755, 766, 776, 787, 798, 808, 818, 828, 838, 848, 858, 869, 880, 890, 904, 915, 926, 937, 948, 959, 970, 980, 991, 1001, 1011, 1021, 1031, 1042, 1052, 1062, 1072, 1082, 1093, 1103, 1114, 1115, 1125, 1136, 1146, 1156, 1167, 1179, 1190, 1201, 1213, 1233, 1244, 1256, 1267, 1278, 1289, 1305, 1320, 1330, 1340, 1351, 1366, 1377, 1398, 1409, 1419, 1454, 1468, 1479, 1490, 1501, 1512, 1523, 1534, 1545, 1556, 1567, 1577, 1587, 1598, 1610, 1620, 1630, 1641, 1651, 1663, 1673, 1683, 1693, 1704, 1714, 1724, 1734, 1744, 1754, 1767, 1780, 1790, 1802, 1813, 1823, 1833, 1843, 1844, 1854, 1865, 1875, 1885, 1896, 1907, 1919, 1929, 1941, 1954, 1964, 1979, 1989, 2031, 2041, 2052, 2063, 2073, 2083, 2093, 2103, 2115, 2129, 2139, 2149, 2165, 2175, 2185, 2195, 2207

In [644]:
df_graph.sort_index(inplace = True) # 경고가 자꾸 떠서...
print(f'총 의원 명수: {len(relation_state)}') # 총 307,707명의 의원,,,
print(f'총 법안 개수: {len(idx)}') # 총 22,956개의 데이터...
cnt = 0
for i in range(len(idx)-1):
    lst = relation_state['이름'][idx[i]:idx[i+1]].copy().sort_values().reset_index(drop = True)
    for j in range(len(lst)-1): 
        for k in range(j+1, len(lst)):
            df_graph.loc[(lst[j], lst[k]), [relation_state['상태'][idx[i]]]] += 1
    print(cnt, end = ' ')
    cnt+=1
    if cnt % 1000 == 0:
        print()
df_graph.to_csv('./raw_data/graph_edge.csv')

총 의원 명수: 307706
총 법안 개수: 22955
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 

In [ ]:
graph_edge = pd.read_csv('./raw_data/graph_edge.csv')
graph_edge.head(3)

In [ ]:
nx_G = nx.from_pandas_edgelist(graph_edge, source = '의원A', target = '의원B', create_using = nx.Graph())
nx_G

In [ ]:
nx_G.number_of_edges(), nx_G.number_of_nodes()

1. top 10% 국회의원이 얼마나 전체 상호작용의 얼마에 참여하는가?

In [ ]:
df['상호작용'].sort_values(ascending = False)

In [ ]:
# 상위 10% -> 20%
df['상호작용'].sort_values(ascending = False)[:30].sum() / df['상호작용'].sum()

In [ ]:
# 하위 10% -> 3%
df['상호작용'].sort_values()[:30].sum() / df['상호작용'].sum()

2. degree of separation?

In [ ]:
dos = sorted(list(graph_edge.상호작용.value_counts()))
print(dos, sep = ' ') # 어...? degree of separation의 평균은 "1"...이다...

3. 평균적으로 얼마나 많은 사람들이 법안에 참여하는지?

In [ ]:
law = []
for i in range(len(idx)-1):
  law.append(idx[i+1]-idx[i])
print(law, sep = ' ')

In [ ]:
sorted(law, reverse = True)

In [ ]:
relation_state[:][1110:1120]

In [ ]:
state[state['인덱스'] == 'PRC_I2N3X1L0I1H3H1V7X5J0K2H3V7I5D8']

In [ ]:
pd.Series(law).describe()

In [ ]:
np.percentile(law, [90, 95, 99])

In [ ]:
cdf = np.arange(1, len(np.sort(law)) + 1) / len(np.sort(law))
plt.plot(np.sort(law), cdf)

4. 초당적 협력

In [ ]:
rel = relation_state.copy()
rel.set_index(['인덱스'], inplace = True)
rel.head(3)

In [ ]:
chodang = []
rel_idx = rel.index.unique()
rel_tmp = rel[:]['정당명'].copy()
cnt = 0
for i in rel_idx:
  if type(rel_tmp[i]) != str:
    rel_tmp[i].unique()
    chodang.append(rel_tmp[i].unique())
  else:
    chodang.append(list(rel_tmp[i]))
  print(cnt, end = ' ')
  cnt += 1
print(chodang, sep = ' ')

In [ ]:
S = pd.Series(chodang)
S.value_counts()

5. 의원

In [ ]:
df_temp = pd.merge(member[['이름','정당명']], df, how = 'inner', on = '이름')
df_temp.head(3)

In [ ]:
dang = df_temp.groupby('정당명').mean()
dang['pos%'] = dang['positive'] / dang['상호작용']
dang['neu%'] = dang['neutral'] / dang['상호작용']
dang['neg%'] = dang['negative'] / dang['상호작용']
dang['rem%'] = dang['remain'] / dang['상호작용']
dang

야당일 수록 계류 대신 의결안이 더 많을 것이다.
그렇진 않은걸

In [ ]:
df_temp2 = pd.merge(member[['이름','위원회']], df, how = 'inner', on = '이름')
df_temp2.head(3)

In [ ]:
commi = df_temp2.groupby('위원회').mean()
commi['pos%'] = commi['positive'] / commi['상호작용']
commi['neu%'] = commi['neutral'] / commi['상호작용']
commi['neg%'] = commi['negative'] / commi['상호작용']
commi['rem%'] = commi['remain'] / commi['상호작용']
commi

많은 위원회에 속해 있을 수록 입법 활동을 더 적극적으로 할 것이다.
그건 또 아님